In [51]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,LSTM,Embedding,Bidirectional
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [2]:
# reading the dataset
data=pd.read_csv('train.csv')

In [3]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
# droping the rows which has the null values
newData = data.dropna(axis=0)

In [25]:
# taking the information of the null values
newData.isnull().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
id        18285 non-null bool
title     18285 non-null bool
author    18285 non-null bool
text      18285 non-null bool
label     18285 non-null bool
dtypes: bool(5)
memory usage: 232.1 KB


In [31]:
# resetting the index of the dataframe
newData.reset_index(inplace=True)

In [32]:
# initializing the stopwords and the Snowball stemmer
stop = stopwords.words('english')
stem = SnowballStemmer('english')

In [33]:
stem.stem('tasty')

'tasti'

In [34]:

corpus = []
for i in tqdm(range(0,len(newData['title']))):
    # removing all the function excdept the charecter for the each title
    reviews = re.sub('[^a-zA-Z]',' ',newData['title'][i])
    # converting the each title to the lower case
    reviews = reviews.lower()
    # splitting the each title based on the space
    reviews = reviews.split()
    
    # removing the stopwords and stemming for each word of each title
    reviews = [stem.stem(words) for words in reviews if not words in stop]
    # joining each word of the title
    reviews=' '.join(reviews)
    # joing all the title
    corpus.append(reviews)

100%|██████████████████████████████████| 18285/18285 [00:07<00:00, 2596.43it/s]


In [40]:
# one hot encoding each title and listing all the one hot encoded title
oneHotData=[one_hot(dat,5000) for dat in corpus ]

In [44]:
# padding alll the sequence
# pad_sequence( data , maxium length it should padd , where the padding should done)
paddedData = pad_sequences(oneHotData,maxlen=40,padding='pre')

In [55]:
# splitting the data
xtrain,xtest,ytrain,ytest=train_test_split(paddedData,newData['label'],test_size=0.3)

In [ ]:
# initializing the Sequential model
model=Sequential()
#creating the embedding layer 
# Embedding(input rows length,output length ,inut column length)
model.add(Embedding(18285,32,input_length=40))
# using the bidirectional LSTM
# for getting the bidirectional LSTM we using the bidirectional and the LSTM
# this has the 100 layers
model.add(Bidirectional(LSTM(100)))
# output sigmoid unit
model.add(Dense(1,activation='sigmoid'))
# compile(which optimizer want to use,)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [59]:
model.fit(xtrain,ytrain,batch_size=128,epochs=10,validation_data=(xtest,ytest))

C:\Users\sheik\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 12799 samples, validate on 5486 samples
Epoch 1/10
12799/12799 [==============================] - 89s 7ms/step - loss: 0.4516 - accuracy: 0.7753 - val_loss: 0.2350 - val_accuracy: 0.9048
Epoch 2/10
12799/12799 [==============================] - 86s 7ms/step - loss: 0.1670 - accuracy: 0.9330 - val_loss: 0.1936 - val_accuracy: 0.9182
Epoch 3/10
12799/12799 [==============================] - 92s 7ms/step - loss: 0.1121 - accuracy: 0.9591 - val_loss: 0.2014 - val_accuracy: 0.9222
Epoch 4/10
12799/12799 [==============================] - 88s 7ms/step - loss: 0.0830 - accuracy: 0.9720 - val_loss: 0.2515 - val_accuracy: 0.9085
Epoch 5/10
12799/12799 [==============================] - 88s 7ms/step - loss: 0.0606 - accuracy: 0.9807 - val_loss: 0.2540 - val_accuracy: 0.9162
Epoch 6/10
12799/12799 [==============================] - 87s 7ms/step - loss: 0.0479 - accuracy: 0.9851 - val_loss: 0.2915 - val_accuracy: 0.9185
Epoch 7/10
12799/12799 [==============================] - 87s 7ms/ste